# 爬取知网相关主题文献信息-1031更新，主要更新内容：①只获取学术期刊；②只获取参考文献中的学术期刊；③自动下载pdf到浏览器默认下载文件夹"C:/Users/用户名/Downloads"并命名为文献名。请使用此代码。

In [ ]:
代码使用说明1031：
①下载适合自己浏览器的驱动并放到和代码文件相同的文件夹。下面是获知符合自己版本的驱动器及下载方法：
1）获知您的电脑上的 Edge 或 Chrome 浏览器版本，以便下载相应的驱动程序，您可以按照以下步骤操作：
    对于 Microsoft Edge:
    打开 Microsoft Edge。
    点击屏幕右上角的三个水平点（菜单按钮）。
    选择 "帮助和反馈" 下的 "关于 Microsoft Edge"。
    您将在此页面上看到浏览器的当前版本号。

    对于 Google Chrome:
    打开 Google Chrome。
    点击屏幕右上角的三个垂直点（菜单按钮）。
    选择 "帮助" 下的 "关于 Google Chrome"。
    您将在此页面上看到浏览器的当前版本号。

2）知道了浏览器的版本后，您可以访问浏览器的官方驱动下载页面来获取相应版本的驱动：
对于 Microsoft Edge：https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
对于 Google Chrome：https://sites.google.com/a/chromium.org/chromedriver/downloads

下载对应的驱动版本，并和代码文件放置在同一文件夹。

②您使用此代码时需要修改的地方：
1）修改您需要的查询主题，具体代码位置：
    theme = "公交导向开发" #请修改公交导向开发为您的关键词
2）修改属于您的用户名文件夹
    download_path = "C:/Users/Ng_HC/Downloads"  # 更改为你的下载文件夹路径，您需要将Ng_HC修改为您的用户名
    
    对于 Edge 和 Google Chrome 浏览器，查看浏览器的默认下载文件夹的步骤如下：
    
    对于 Microsoft Edge:
    打开 Microsoft Edge。
    点击屏幕右上角的三个水平点（菜单按钮）。
    选择 "设置"。
    在设置页面的左侧菜单中，选择 "下载"。
    在 "下载" 选项中，您将看到一个标记为 "下载位置" 的部分，该部分会显示浏览器的默认下载文件夹。
    
    对于 Google Chrome:
    打开 Google Chrome。
    点击屏幕右上角的三个垂直点（菜单按钮）。
    选择 "设置"。
    在设置页面的搜索框中，输入 "下载"。
    在搜索结果中，找到 "下载位置" 部分，它会显示浏览器的默认下载文件夹。
3）代码获取的数据存在名为"CNKI_{您的关键词}.xlsx"的文件中。
    

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import openpyxl
import os
from shutil import move

# 创建一个新的Excel工作簿
workbook = openpyxl.Workbook()
sheet = workbook.active

# 写入列名
column_names = ['title', 
                'authors', 
                'institute', 
                'date', 
                'source', 
                'database', 
                'cite', 
                'download', 
                'keywords', 
                'abstract', 
                'url', 
                'journal']
for col_idx, column_name in enumerate(column_names, start=1):
    sheet.cell(row=1, column=col_idx, value=column_name)


def open_page(driver, theme):
    # 打开页面
    driver.get("https://www.cnki.net")
    # 传入关键字
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '''//*[@id="txt_SearchText"]'''))).send_keys(theme)
    # 点击搜索
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[1]/input[2]"))).click()
#     time.sleep(3)

    # 点击切换中文文献
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[1]/div[2]/div/div/div/div/a[1]"))).click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[1]/div[2]/div/div/ul/li[1]"))).click()
#     time.sleep(3)
    # 获取总文献数和页数
    res_unm = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, "/html/body/div[2]/div[2]/div[2]/div[2]/div/div[1]/div/div[1]/span[1]/em"))).text
    # 去除千分位里的逗号
    res_unm = int(res_unm.replace(",", ''))
    page_unm = int(res_unm / 20) + 1
    print(f"共找到 {res_unm} 条结果, {page_unm} 页。")
    return res_unm



def crawl(driver, papers_need, theme):
    # 赋值序号, 控制爬取的文章数量
    count = 1

    # 当爬取数量小于需求时，循环网页页码
    while count <= papers_need:
        # 等待加载完全，休眠3S
        time.sleep(3)

        title_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fz14")))
        # 循环网页一页中的条目
        for i in range(len(title_list)):
            try:
                term = count % 20  # 本页的第几个条目
                title_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[2]"
                author_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[3]"
                source_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[4]"
                date_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[5]"
                database_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[6]"
                cite_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[7]"
                download_xpath = f"/html/body/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[1]/div/div/table/tbody/tr[{term}]/td[8]"


                # 点击条目
                title_list[i].click()
                # 获取driver的句柄
                n = driver.window_handles
                # driver切换至最新生产的页面
                driver.switch_to.window(n[-1])

                download_button_xpath = '//*[@id="DownLoadParts"]/div[2]/ul/li[4]'
                

                    
                try:
                    download_button = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, download_button_xpath))
                    )
                    download_button.click()

                    # 等待下载完成，这个时间可能需要根据实际情况进行调整
                    time.sleep(3)
                    # 下载PDF并重命名
                    download_path = "C:/Users/Ng_HC/Downloads"  # 更改为你的下载文件夹路径
                    new_filename = os.path.join(download_path, title + ".pdf")

                    # 假设文件被下载到默认的Downloads文件夹
                    latest_file = max([os.path.join(download_path, f) for f in os.listdir(download_path)],
                                      key=os.path.getctime)

                    if os.path.splitext(latest_file)[1] == ".pdf":
                        move(latest_file, new_filename)
                    print(f"文章 {title} 已经储存到文件夹 {download_path} ")
                except:
                    print(f"文章 {title} 没有找到下载按钮或下载失败")
                    
                url = driver.current_url
                

                

                                 
            except:
                print(f" 第{count} 条爬取失败\n")
                # 跳过本条，接着下一个
                continue
            finally:
                # 如果有多个窗口，关闭第二个窗口， 切换回主页
                n2 = driver.window_handles
                if len(n2) > 1:
                    driver.close()
                    driver.switch_to.window(n2[0])
                # 计数,判断需求是否足够
                count += 1
                if count == papers_need: break

    

if __name__ == "__main__":
    # get直接返回，不再等待界面加载完成
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["pageLoadStrategy"] = "none"

    import pandas as pd

    # 读取Excel文件
    df = pd.read_excel('D:\知识图谱\测试100.xlsx')

    # 遍历'title'列的每一行
    for theme in df['title']:
        options = webdriver.EdgeOptions()
        options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        driver = webdriver.Edge(options=options)
        
        res_unm = int(open_page(driver, theme))
        papers_need = res_unm
        crawl(driver, papers_need, theme)
        time.sleep(3)
        driver.close()


共找到 1 条结果, 1 页。
 第1 条爬取失败

共找到 1 条结果, 1 页。
 第1 条爬取失败

共找到 2 条结果, 1 页。
 第1 条爬取失败

 第2 条爬取失败

 第3 条爬取失败

共找到 73 条结果, 4 页。
 第1 条爬取失败

 第2 条爬取失败

 第3 条爬取失败

 第4 条爬取失败

 第5 条爬取失败

 第6 条爬取失败

 第7 条爬取失败

 第8 条爬取失败

 第9 条爬取失败

 第10 条爬取失败



WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: MicrosoftEdge=119.0.2151.58)
Stacktrace:
	GetHandleVerifier [0x00007FF6BF2E25B2+60402]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6BF267302+253666]
	(No symbol) [0x00007FF6BF039C99]
	(No symbol) [0x00007FF6BF02073D]
	(No symbol) [0x00007FF6BF021D4E]
	(No symbol) [0x00007FF6BF03A224]
	(No symbol) [0x00007FF6BF0107A5]
	(No symbol) [0x00007FF6BF0B7BC8]
	(No symbol) [0x00007FF6BF0ABA50]
	(No symbol) [0x00007FF6BF07C585]
	(No symbol) [0x00007FF6BF07B993]
	(No symbol) [0x00007FF6BF07CD14]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF6BF4C3454+1161924]
	(No symbol) [0x00007FF6BF0FE5F6]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6BF1BB083+37459]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6BF1B2C4D+3613]
	Microsoft:

# 爬取知网相关主题文献信息-0626更新，请注意，由于知网更新下面此代码不可用，仅作存档，具体使用代码请看第一段代码。

In [5]:
driver = webdriver.Edge(options=options)

In [8]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import openpyxl

# 创建一个新的Excel工作簿
workbook = openpyxl.Workbook()
sheet = workbook.active

# 写入列名
column_names = ['title', 
                'authors', 
                'institute', 
                'date', 
                'source', 
                'database', 
                'cite', 
                'download', 
                'keywords', 
                'abstract', 
                'url', 
                'journal', 
                'dissertation-d', 
                'dissertation-m', 
                'conference-c', 
                'conference-i', 
                'yearbook', 
                'newpaper', 
                'journal-w', 
                'book', 
                'patent', 
                'stand', 
                'crldeng']
for col_idx, column_name in enumerate(column_names, start=1):
    sheet.cell(row=1, column=col_idx, value=column_name)


def open_page(driver, theme):
    # 打开页面
    driver.get("https://www.cnki.net")
    # 传入关键字
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '''//*[@id="txt_SearchText"]'''))).send_keys(theme)
    # 点击搜索
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[1]/input[2]"))).click()
    time.sleep(3)

    # 点击切换中文文献
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[1]/div/div/div/a[1]"))).click()
    time.sleep(3)
    # 获取总文献数和页数
    res_unm = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/form/div/div[1]/div[1]/span[1]/em"))).text
    # 去除千分位里的逗号
    res_unm = int(res_unm.replace(",", ''))
    page_unm = int(res_unm / 20) + 1
    print(f"共找到 {res_unm} 条结果, {page_unm} 页。")
    return res_unm

def crawl(driver, papers_need, theme):
    # 赋值序号, 控制爬取的文章数量
    count = 1

    # 当爬取数量小于需求时，循环网页页码
    while count <= papers_need:
        # 等待加载完全，休眠3S
        time.sleep(3)

        title_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fz14")))
        # 循环网页一页中的条目
        for i in range(len(title_list)):
            try:
                term = count % 20  # 本页的第几个条目
                title_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[2]"
                author_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[3]"
                source_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[4]"
                date_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[5]"
                database_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[6]"
                cite_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[7]"
                download_xpath = f"/html/body/div[3]/div[2]/div[2]/div[2]/form/div/table/tbody/tr[{term}]/td[8]"
                title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, title_xpath))).text
                authors = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, author_xpath))).text
                source = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, source_xpath))).text
                date = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, date_xpath))).text
                database = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, database_xpath))).text
                cite = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, cite_xpath))).text
                download = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, download_xpath))).text

                # 点击条目
                title_list[i].click()
                # 获取driver的句柄
                n = driver.window_handles
                # driver切换至最新生产的页面
                driver.switch_to.window(n[-1])
#                 time.sleep(3)
                # 开始获取页面信息
                title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,"/html/body/div[2]/div[1]/div[3]/div/div/div[3]/div/h1"))).text
                authors = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH ,"/html/body/div[2]/div[1]/div[3]/div/div/div[3]/div/h3[1]"))).text
                institute = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
                    (By.XPATH, "/html/body/div[2]/div[1]/div[3]/div/div/div[3]/div/h3[2]"))).text
                abstract = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "abstract-text"))).text
                
                

                try:
                    # 获取当前页面的高度
                    def get_page_height(driver):
                        return driver.execute_script("return document.body.scrollHeight")

                    # 滚动到页面底部
                    def scroll_to_bottom(driver):
                        while True:
                            initial_height = get_page_height(driver)
                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            time.sleep(2)
                            new_height = get_page_height(driver)
                            if new_height == initial_height:
                                break
    
                    element_xpaths = ['//*[@id="nxgp-kcms-data-ref-references-journal"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-dissertation-d"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-dissertation-m"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-conference-c"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-conference-i"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-yearbook"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-newpaper"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-journal-w"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-book"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-patent"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-stand"]/ul',
                                      '//*[@id="nxgp-kcms-data-ref-references-crldeng"]/ul']  # 存储元素XPath的数组
                    next_button_xpaths = ['//*[@id="nxgp-kcms-data-ref-references-journal-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-dissertation-d-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-dissertation-m-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-conference-c-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-conference-i-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-yearbook-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-newpaper-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-journal-w-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-book-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-patent-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-stand-page"]/a[@class="next"]',
                                          '//*[@id="nxgp-kcms-data-ref-references-crldeng-page"]/a[@class="next"]']  # 存储下一页按钮XPath的数组

                    elements_dict = {
                            'journal': [],
                            'dissertation-d': [],
                            'dissertation-m': [],
                            'conference-c': [],
                            'conference-i': [],
                            'yearbook': [],
                            'newpaper': [],
                            'journal-w': [],
                            'book': [],
                            'patent': [],
                            'stand': [],
                            'crldeng': []
                        }

                    def get_all_elements(driver):
                        keys = list(elements_dict.keys())
                        for i in range(len(element_xpaths)):
                            element_xpath = element_xpaths[i]
                            next_button_xpath = next_button_xpaths[i]
                            elements = []  # 用于存储当前容器的所有元素

                            while True:
                                # 获取当前页的ul元素
                                try:
                                    element = WebDriverWait(driver, 10).until(
                                        EC.presence_of_element_located((By.XPATH, element_xpath)))
                                    element_text = element.text
                                    # 在elements列表中添加element_text
                                    if elements:
                                        elements[-1] += '\n' + element_text  # 在上一个元素的末尾添加换行符和新的element_text
                                    else:
                                        elements.append(element_text)  # 第一个元素直接添加到列表中

                                    print("参考文献:", elements)
                                    try:
                                        # 判断是否有下一页
                                        next_button = driver.find_element(By.XPATH, next_button_xpath)
                                        if next_button.get_attribute("class") == "next":  # 如果下一页按钮可点击，则表示有下一页
                                            next_button.click()
                                            # 等待页面加载完成，不等待会一直加载第一页
                                            WebDriverWait(driver, 10).until(
                                                EC.presence_of_element_located((By.XPATH, element_xpath)))
                                        else:
                                            break
                                    except:
                                        print("最后一页")
                                        break
                                except:
#                                     print("未找到元素:", element_xpath)
                                    break

                            # 将元素的文本内容添加到对应的字典项
                            elements_dict[keys[i]] = elements
                        return elements_dict
                    
                    
                    # 滚动到页面底部
                    scroll_to_bottom(driver)

                    # 查看字典是否正常
                    elements_dict = get_all_elements(driver)
                    print("字典:", elements_dict)

                    keywords = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "keywords"))).text[:-1]
                except:
                    keywords = '无'
                    
                    
                url = driver.current_url
                
                # 写入单行数据
                row_data = [title, authors, institute, date, source, database, cite, download, keywords, abstract, url]
                for column in column_names[11:]:  # 对应于 elements_dict 的键
                    # 把 elements_dict 中的列表转换成字符串，然后添加到 row_data 中
                    row_data.append(','.join(elements_dict[column]))

                sheet.append(row_data)
                # 保存Excel文件
                workbook.save(f'CNKI_{theme}.xlsx')

                 

                
            except:
                print(f" 第{count} 条爬取失败\n")
                # 跳过本条，接着下一个
                continue
            finally:
                # 如果有多个窗口，关闭第二个窗口， 切换回主页
                n2 = driver.window_handles
                if len(n2) > 1:
                    driver.close()
                    driver.switch_to.window(n2[0])
                # 计数,判断需求是否足够
                count += 1
                if count == papers_need: break

        # 切换到下一页
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@id='PageNext']"))).click()
    

if __name__ == "__main__":
    # get直接返回，不再等待界面加载完成
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["pageLoadStrategy"] = "none"

    # 设置谷歌驱动器的环境
    options = webdriver.EdgeOptions()
    # 设置chrome不加载图片，提高速度
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    # # 设置不显示窗口
    # options.add_argument('--headless')
    # 创建一个谷歌驱动器
    driver = webdriver.Edge(options=options)
    # 设置搜索主题
    theme = "TOD"
    # 设置所需篇数
#     papers_need = 100
    
    res_unm = int(open_page(driver, theme))
    # 设置所需篇数
    papers_need = res_unm
    
    # 判断所需是否大于总篇数
#     papers_need = papers_need if (papers_need <= res_unm) else res_unm
    # 不爬取无参考文献的文章
    crawl(driver, papers_need, theme)

    # 关闭浏览器
    driver.close()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64E3799D2+63682]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF64E301922+267714]
	(No symbol) [0x00007FF64E0BAC0E]
	(No symbol) [0x00007FF64E10B4D7]
	(No symbol) [0x00007FF64E10B5E8]
	(No symbol) [0x00007FF64E149A27]
	(No symbol) [0x00007FF64E12C88F]
	(No symbol) [0x00007FF64E0FEC51]
	(No symbol) [0x00007FF64E146A81]
	(No symbol) [0x00007FF64E12C623]
	(No symbol) [0x00007FF64E0FDA7A]
	(No symbol) [0x00007FF64E0FCD6B]
	(No symbol) [0x00007FF64E0FE204]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF64E5467A9+1233737]
	(No symbol) [0x00007FF64E17B274]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF64E2432AA+33498]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF64E23BDE9+3609]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF64E545544+1229028]
	Microsoft::Applications::Events::ILogConfiguration::opera